## Spark SQL

### Code to be executed before lecture

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col
import requests

In [ ]:
spark = (SparkSession.builder.appName("cs544")
         .master("spark://boss:7077")
         .config("spark.executor.memory", "512M")
         .config("spark.sql.warehouse.dir", "hdfs://nn:9000/user/hive/warehouse")
         .enableHiveSupport()
         .getOrCreate())

#### If you did not bring the Spark cluster down, you don't have to execute the below code. If you did bring it down, then please execute the below cells.

In [ ]:
! wget https://ms.sites.cs.wisc.edu/cs544/data/sf.zip

In [ ]:
!unzip sf.zip

In [ ]:
!ls -lah

In [ ]:
!hdfs dfs -cp sf.csv hdfs://nn:9000/sf.csv

In [ ]:
df = (spark.read.format("csv")
      .option("header", True)
      .option("inferSchema", True)
      .load("hdfs://nn:9000/sf.csv"))

In [ ]:
cols = [col(c).alias(c.replace(" ", "_")) for c in df.columns]
df.select(cols).write.format("parquet").mode("overwrite").save("hdfs://nn:9000/sf.parquet")
df = spark.read.format("parquet").load("hdfs://nn:9000/sf.parquet")

In [ ]:
!hdfs dfs -rm hdfs://nn:9000/sf.csv

### Lecture starts here

### HIVE View

Let's rename "Neighborhooods_-_Analysis_Boundaries" to "area".

In [ ]:
df.withColumnRenamed("Neighborhooods_-_Analysis_Boundaries", "area").createOrReplaceTempView("calls")

In [ ]:
spark.sql("SHOW TABLES").show()

In [ ]:
spark.sql("SELECT * FROM calls LIMIT 3").toPandas()

### HIVE table

In [ ]:
spark.sql("""
SELECT *
FROM calls
WHERE Call_Type LIKE 'Odor%'
""").write.mode("overwrite").saveAsTable("stinky")

In [ ]:
spark.sql("SHOW TABLES").show()

In [ ]:
spark.sql("SELECT * FROM stinky LIMIT 3").toPandas()

### HIVE data on HDFS

Where is it located?

```python
spark = (SparkSession.builder.appName("cs544")
         .master("spark://boss:7077")
         .config("spark.executor.memory", "512M")
         .config("spark.sql.warehouse.dir", "hdfs://nn:9000/user/hive/warehouse")
         .enableHiveSupport()
         .getOrCreate())
```

In [ ]:
!hdfs dfs -ls hdfs://nn:9000/user/hive/warehouse/stinky/

In [ ]:
df.rdd.getNumPartitions()

### Number of partitions: writing vs reading data

In [ ]:
spark.sql("SELECT * FROM calls").rdd.getNumPartitions()

In [ ]:
spark.sql("SELECT * FROM stinky").rdd.getNumPartitions()

### Create DataFrame from HIVE view or table

In [ ]:
spark.table("calls")

In [ ]:
spark.table("stinky")

### Grouping

#### What are the unique area column values?

In [ ]:
spark.sql("SELECT DISTINCT area FROM calls").collect()

#### How many calls are there per area?

In [ ]:
pandas_df = spark.sql("""
SELECT area, COUNT(*) as count
FROM calls
GROUP BY area
ORDER BY count DESC
""").toPandas()
pandas_df

In [ ]:
pandas_df.set_index("area").plot.bar()

#### How many calls are there per groups/type?

In [ ]:
spark.sql("""
SELECT Call_Type_Group, Call_Type, COUNT(*) as count
FROM calls
GROUP BY Call_Type_Group, Call_Type
""").toPandas().head()

#### For each call group, what percentage of calls are represented by the biggest type?

In [ ]:
spark.sql("""
SELECT Call_Type_Group, MAX(count) / SUM(count)
FROM (
    SELECT Call_Type_Group, Call_Type, COUNT(*) as count
    FROM calls
    GROUP BY Call_Type_Group, Call_Type
)
GROUP BY Call_Type_Group
""").toPandas()

Let's use DataFrame API to solve the same question.

In [ ]:
(spark.table("calls")
 .groupby("Call_Type_Group", "Call_Type")
 .count()
 .groupby("Call_Type_Group")
 .agg(expr("MAX(count) / SUM(count)").alias("perc"))
).toPandas()

### Window functions

#### What are three smallest call numbers for each area?

In [ ]:
spark.sql("""
SELECT area, Call_Number, row_number() OVER (PARTITION BY area ORDER BY Call_Number ASC) AS rownum
FROM calls
""").where("rownum <= 3").toPandas()

### Holidays dataset

In [ ]:
!hdfs dfs -cp holidays2.csv hdfs://nn:9000/holidays2.csv

In [ ]:
(spark.read
 .format("csv")
 .option("inferSchema", True)
 .option("header", True)
 .load("hdfs://nn:9000/holidays2.csv")
 .createOrReplaceTempView("holidays")
)

In [ ]:
spark.table("holidays")

### Joining the SF fire data with holidays data

SQL version

In [ ]:
spark.sql("""
SELECT *
FROM calls
LIMIT 5
""").toPandas()

In [ ]:
spark.sql("""
SELECT *
FROM holidays
LIMIT 5
""").toPandas()

In [ ]:
spark.sql("""
SELECT *
FROM calls
INNER JOIN holidays ON calls.Call_Date = holidays.date
LIMIT 5
""").toPandas()

DataFrame version

In [ ]:
# Create the DataFrames
calls = spark.table("calls")
holidays = spark.table("holidays")

In [ ]:
# this doesn't trigger compute in Spark unlike pandas
calls["Call_Date"] == holidays["date"]

In [ ]:
calls.join(holidays, on=calls["Call_Date"] == holidays["date"], how="inner").limit(5).toPandas()

#### How many calls on each kind of holiday?

In [ ]:
(calls
 .join(holidays, on=calls["Call_Date"] == holidays["date"], how="inner")
 .groupby("holiday")
 .count()
 .orderBy("count")
 .toPandas())

#### What percent of fire dept calls are on holidays?

In [ ]:
(calls
 .join(holidays, on=calls["Call_Date"] == holidays["date"], how="left")
 .agg(expr("COUNT(holiday) / COUNT(*)"))
 .toPandas())

### Web server REST API

Documentation: https://spark.apache.org/docs/latest/monitoring.html#rest-api

```
http://localhost:4040/api/v1/applications
http://localhost:4040/api/v1/applications/{app_id}/executors
# look for "totalTasks"
```

#### Appplications information
```
http://localhost:4040/api/v1/applications
```

In [ ]:
r = requests.get("http://localhost:4040/api/v1/applications")
r.raise_for_status()
r.json()

Extracting app id.

In [ ]:
app_id = r.json()[0]["id"]
app_id

#### Executors information

For example, how much work each executor has done

In [ ]:
r = requests.get(f"http://localhost:4040/api/v1/applications/{app_id}/executors")
r.raise_for_status()
r.json()

#### How many total tasks have been run by each executor?

In [ ]:
[exec["totalTasks"] for exec in r.json()]

### Caching

Let's sample the data, create a single partition and try some caching.

In [ ]:
# uses StorageLevel as "MEMORY_ONLY"
df = spark.table("calls").sample(True, 0.01).repartition(1).cache()

Let's count the number of rows in our sample.

In [ ]:
df.count()

Let's take a look at total tasks executed by each executor.

In [ ]:
r = requests.get(f"http://localhost:4040/api/v1/applications/{app_id}/executors")
r.raise_for_status()
[exec["totalTasks"] for exec in r.json()]

In [ ]:
# Repeating count computation 30 times
for i in range(30):
    df.count()

In [ ]:
r = requests.get(f"http://localhost:4040/api/v1/applications/{app_id}/executors")
r.raise_for_status()
[exec["totalTasks"] for exec in r.json()]

How can we have both executors do the work when we do caching? We need to use `StorageLevel` as `MEMORY_ONLY_2`. 
<br>**Try it by yourself using `persist` method instead of `cache` method.**

### Hash partitioning

What is a hash function?
- takes anything (e.g., just some bytes containing some data)
- returns a number (deteriministic, but ideally not with an obvious pattern)

In [ ]:
hash(b"a")

In [ ]:
print(hash(b"a"))
print(hash(b"b"))
print(hash(b"c"))
print(hash(b"d"))
print(hash(b"e"))
print(hash(b"f"))

We can use modulo operator (`%`) to determine what hash partition a particular value should go into.

For example, if we need 5 partitions:

In [ ]:
print(hash(b"a") % 5)
print(hash(b"b") % 5)
print(hash(b"c") % 5)
print(hash(b"d") % 5)
print(hash(b"e") % 5)
print(hash(b"f") % 5)

In [ ]:
random_string = "aaaabbbefghihijkllmlm"

In [ ]:
partitions = [[], [], [], [], []]
for letter in random_string:
    partition_idx = hash(letter) % len(partitions)
    partitions[partition_idx].append(letter)
partitions

#### Spark execution explanation

`.explain()` or `.explain("formatted")`

In [ ]:
spark.sql("""
SELECT Call_Type, COUNT(*) as count
FROM calls
GROUP BY Call_Type
""").explain()

In [ ]:
spark.sql("""
SELECT Call_Type, COUNT(*) as count
FROM calls
GROUP BY Call_Type
""").explain("formatted")